<a href="https://colab.research.google.com/github/Kekc/dogsML/blob/main/dogsml/colab/dog_breeds_77.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar

--2022-08-02 14:18:37--  http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
Resolving vision.stanford.edu (vision.stanford.edu)... 171.64.68.10
Connecting to vision.stanford.edu (vision.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 793579520 (757M) [application/x-tar]
Saving to: ‘images.tar’

images.tar          100%[===================>] 756.82M  50.6MB/s    in 9.9s    

2022-08-02 14:18:47 (76.8 MB/s) - ‘images.tar’ saved [793579520/793579520]



In [ ]:
import tarfile
tar_file = tarfile.open("./images.tar", 'r')
tar_file.extractall("tmp/")
tar_file.close()



In [ ]:
import os
images_path = "tmp/Images"

for item in os.listdir(images_path):
  new_item = item[item.find("-")+1:]
  new_item = new_item.capitalize()
  os.rename(os.path.join(images_path, item), os.path.join(images_path, new_item))


In [ ]:
import os
os.listdir("tmp/Images")

['Schipperke',
 'Lhasa',
 'Bedlington_terrier',
 'Saint_bernard',
 'Giant_schnauzer',
 'Maltese_dog',
 'Bouvier_des_flandres',
 'Papillon',
 'Welsh_springer_spaniel',
 'German_shepherd',
 'Silky_terrier',
 'Miniature_pinscher',
 'Shetland_sheepdog',
 'Beagle',
 'Lakeland_terrier',
 'Great_pyrenees',
 'Irish_setter',
 'Blenheim_spaniel',
 'Cardigan',
 'Black-and-tan_coonhound',
 'Kuvasz',
 'African_hunting_dog',
 'Walker_hound',
 'Redbone',
 'Chesapeake_bay_retriever',
 'Affenpinscher',
 'Sussex_spaniel',
 'Irish_wolfhound',
 'Scottish_deerhound',
 'English_foxhound',
 'Bluetick',
 'Basenji',
 'Airedale',
 'Italian_greyhound',
 'Basset',
 'Miniature_poodle',
 'Malinois',
 'Groenendael',
 'Norwegian_elkhound',
 'Border_terrier',
 'Briard',
 'Flat-coated_retriever',
 'Norwich_terrier',
 'Kerry_blue_terrier',
 'Shih-tzu',
 'Wire-haired_fox_terrier',
 'Rottweiler',
 'Entlebucher',
 'Mexican_hairless',
 'Borzoi',
 'Cocker_spaniel',
 'Tibetan_mastiff',
 'Gordon_setter',
 'English_springer',
 

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.optimizers import Adam

In [ ]:
img_height = 224
img_width = 224
batch_size = 32
validation_split = 0.2
seed = 123
num_epochs = 1
data_dir = "tmp/Images"

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)
validation_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

Found 20580 files belonging to 120 classes.
Using 16464 files for training.
Found 20580 files belonging to 120 classes.
Using 4116 files for validation.


In [ ]:
class_names = train_ds.class_names

AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
input_shape = (img_height, img_width, 3)
base_model = tf.keras.applications.MobileNetV3Large(
    input_shape=input_shape,
    include_top=False,
    weights="imagenet",
)

data_augmentation = tf.keras.Sequential([
    tfl.RandomFlip("horizontal"),
    tfl.RandomRotation(0.2),
    tfl.RandomContrast(factor=(0, 0.2)),
    tfl.RandomHeight(factor=(0, 0.3)),
    tfl.RandomWidth(factor=(0, 0.3)),
    tfl.RandomTranslation(
        height_factor=0.3,
        width_factor=0.3,
        fill_mode="nearest",
    ),
    tfl.RandomZoom(
        height_factor=(0, 0.3),
        width_factor=(0, 0.3),
        fill_mode="nearest",
    ),
])

base_model.trainable = False
inputs = tf.keras.Input(shape=input_shape)

x_data = data_augmentation(inputs)
# x_data = inputs
# x_data = tf.keras.applications.mobilenet_v2.preprocess_input(x_data)

x_data = base_model(x_data, training=False)
x_data = tfl.GlobalAveragePooling2D()(x_data)
# x_data = tfl.Dropout(0.2)(x_data)
outputs = tfl.Dense(
    len(class_names),
    activation="softmax",
    kernel_initializer=glorot_uniform(seed=0)
)(x_data)

transfer_model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 1 * 10**-3
transfer_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
num_epochs = 4
history_top = transfer_model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=num_epochs,
)

base_model = transfer_model.layers[2]
base_model.trainable = True
fine_tune_at = 210
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

transfer_model.compile(
    optimizer=Adam(learning_rate=base_learning_rate/10),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

history_base = transfer_model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=num_epochs * 2,
    initial_epoch=history_top.epoch[-1],
)

Epoch 1/4
515/515 [==============================] - 47s 83ms/step - loss: 2.5046 - accuracy: 0.3875 - val_loss: 0.9199 - val_accuracy: 0.7174
Epoch 2/4
515/515 [==============================] - 42s 81ms/step - loss: 1.5851 - accuracy: 0.5622 - val_loss: 0.8048 - val_accuracy: 0.7534
Epoch 3/4
515/515 [==============================] - 41s 79ms/step - loss: 1.4288 - accuracy: 0.6050 - val_loss: 0.7673 - val_accuracy: 0.7617
Epoch 4/4
515/515 [==============================] - 41s 79ms/step - loss: 1.3122 - accuracy: 0.6262 - val_loss: 0.7473 - val_accuracy: 0.7697
Epoch 4/8
515/515 [==============================] - 52s 93ms/step - loss: 1.0250 - accuracy: 0.7036 - val_loss: 0.7224 - val_accuracy: 0.7792
Epoch 5/8
515/515 [==============================] - 47s 91ms/step - loss: 0.9091 - accuracy: 0.7304 - val_loss: 0.7359 - val_accuracy: 0.7707
Epoch 6/8
515/515 [==============================] - 48s 92ms/step - loss: 0.8344 - accuracy: 0.7509 - val_loss: 0.7867 - val_accuracy: 0.7694

In [ ]:
transfer_model.save("dog_breed_colab.h5", save_format="h5")


# New Section